# Add [GDP, Urbanization, Democracy Index, Median Age, Daily Tests] Notebook

Adds the following data:
- GDP in billion USD per country (per US states and China provinces, with plans to add Australian, French, and Canadian provinces as well)
- Urbanization as population percentage per country (per US states and China provinces, with plans to add Australian, French, and Canadian provinces as well)
- Democracy Index per country
- Median Age per country and per US state (with plans to add sub region data for other countries like above)
- Number of Daily tests per US state (we don't believe we will get accurate data for this metric for other subregions)
- We wanted to do Gini Index as well but didn't get to it


Contact ShuliFinley@gmail.com for questions :)

In [1]:
import pandas as pd
import numpy as np
import pickle
import datetime

## _Import and load hopkins data_

In [254]:
# PICKLE_PATH = '../augmented_datasets/pickles/hopkins_conf_gf0904.pkl'
# RESULT_PATH = '../augmented_datasets/pickles/hopkins_conf_gf0904_GDP_urban_tests.pkl'

PICKLE_PATH = '../augmented_datasets/pickles/hopkins_conf_gf0904_GDP_urban_weather.pkl'
RESULT_PATH = '../augmented_datasets/pickles/hopkins_conf_gf0904_GDP_urban_weather_dem_age_dtests_pop.pkl'

In [4]:
hopkins_conf = pd.DataFrame()
with open(PICKLE_PATH, 'rb') as file:
#     hopkins_conf = pickle.load(file) # old pandas versions use this line to load pickle file
    hopkins_conf = pd.read_pickle(file) # updated pandas versions use this line to load pickle file
hopkins_conf

Province_State Country_Region          GDP  \
coordinate           information                                               
(-41.4545, 145.9707) data               Tasmania      Australia  57373.68668   
                     avg_d_RH                NaN            NaN          NaN   
                     avg_d_precip            NaN            NaN          NaN   
                     avg_d_tmp               NaN            NaN          NaN   
                     avg_d_wind              NaN            NaN          NaN   
...                                          ...            ...          ...   
(64.9631, -19.0208)  data                    NaN        Iceland  73191.11632   
                     avg_d_RH                NaN            NaN          NaN   
                     avg_d_precip            NaN            NaN          NaN   
                     avg_d_tmp               NaN            NaN          NaN   
                     avg_d_wind              NaN            NaN          NaN   

                                   Urbanization  avg_m_tmp   avg_m_RH  \
coordinate           information                                        
(-41.4545, 145.9707) data                86.012  15.105195  70.402597   
                     avg_d_RH               NaN        NaN        NaN   
                     avg_d_precip           NaN        NaN        NaN   
                     avg_d_tmp              NaN        NaN        NaN   
                     avg_d_wind             NaN        NaN        NaN   
...                                         ...        ...        ...   
(64.9631, -19.0208)  data                93.813  -1.909459  77.459459   
                     avg_d_RH               NaN        NaN        NaN   
                     avg_d_precip           NaN        NaN        NaN   
                     avg_d_tmp              NaN        NaN        NaN   
                     avg_d_wind             NaN        NaN        NaN   

                                   avg_m_precip  avg_m_wind  Max_Cases  \
coordinate           information                                         
(-41.4545, 145.9707) data                   0.0   18.087013       89.0   
                     avg_d_RH               NaN         NaN        NaN   
                     avg_d_precip           NaN         NaN        NaN   
                     avg_d_tmp              NaN         NaN        NaN   
                     avg_d_wind             NaN         NaN        NaN   
...                                         ...         ...        ...   
(64.9631, -19.0208)  data                   0.0   17.535135     1586.0   
                     avg_d_RH               NaN         NaN        NaN   
                     avg_d_precip           NaN         NaN        NaN   
                     avg_d_tmp              NaN         NaN        NaN   
                     avg_d_wind             NaN         NaN        NaN   

                                    first_7  ... 3/29/2020 3/30/2020  \
coordinate           information             ...                       
(-41.4545, 145.9707) data          1.000000  ...      66.0      66.0   
                     avg_d_RH           NaN  ...      85.0      80.0   
                     avg_d_precip       NaN  ...       0.0       0.0   
                     avg_d_tmp          NaN  ...      15.6      14.8   
                     avg_d_wind         NaN  ...      14.5      24.9   
...                                     ...  ...       ...       ...   
(64.9631, -19.0208)  data          1.588235  ...    1020.0    1086.0   
                     avg_d_RH           NaN  ...      61.0      63.0   
                     avg_d_precip       NaN  ...       0.0       0.0   
                     avg_d_tmp          NaN  ...       6.5       6.2   
                     avg_d_wind         NaN  ...      31.3      31.4   

                                  3/31/2020  4/1/2020  4/2/2020  4/3/2020  \
coordinate           information                                  

## _Preliminary checks on imported data_

In [5]:
# Checking that our starting data has no nan country_region entries in 'data' rows
print('Original df dimensions: {}'.format(hopkins_conf.shape))
check = pd.DataFrame(hopkins_conf.xs('data', level='information', axis=0))
print('Number of \'data\' rows: {}'.format(check.shape[0]))
check = check[check['Country_Region'].isna()]
print('Number of \'data\' rows with NaN Country_Region: {}'.format(check.shape[0]))

Original df dimensions: (5350, 95)
Number of 'data' rows: 1070
Number of 'data' rows with NaN Country_Region: 0


## _Load external datasets_

In [216]:
# ========== GDP BY COUNTRY ========
# source: https://data.worldbank.org/indicator/NY.GDP.PCAP.CD  - in current USD
gdp_country_path = '../external_datasets/GDP_per_capita_countries.csv'
gdp_country = pd.read_csv(gdp_country_path)
gdp_country.columns=['Country_Region', 'GDP']


# ========== US STATES GDP ========
# source: https://en.wikipedia.org/wiki/List_of_U.S._states_by_GDP_per_capita#cite_note-3
# source: https://www.statista.com/statistics/248023/us-gross-domestic-product-gdp-by-state/ 
gdp_us_state_path = '../external_datasets/GDP_per_capita_states.csv'
gdp_us_state = pd.read_csv(gdp_us_state_path)
gdp_us_state.columns = ['Province_State', 'GDP']


# ========== CHINA PROVINCES GDP ========
#source: https://en.wikipedia.org/wiki/List_of_Chinese_administrative_divisions_by_GDP_per_capita
gdp_china_province_path = '../external_datasets/GDP_per_capita_china_provinces.csv'
gdp_china_province = pd.read_csv(gdp_china_province_path)
gdp_china_province.columns = ['Province_State', 'GDP']


# ========== GDP US STATES AND CHINA PROVINCES APPENDED ======== # for convenience
gdp_all_provinces = gdp_us_state.append(gdp_china_province, ignore_index=True)



# ========== URBANIZATION BY COUNTRY ========
# source: http://wdi.worldbank.org/table/4.2# (2018)
urban_country_path = '../external_datasets/urbanization_data.csv'
urban_country = pd.read_csv(urban_country_path)
urban_country.columns=['Country_Region', 'Urbanization']


# ========== US STATES URBANIZATION ========
# source: US census bureau (most updated was is from 2010)
urban_us_state_path = '../external_datasets/urbanization_states.csv'
urban_us_state = pd.read_csv(urban_us_state_path)
urban_us_state.columns = ['Province_State', 'Urbanization']


# ========== CHINA PROVINCES URBANIZATION ========
# source: https://en.wikipedia.org/wiki/Urbanization_in_China
urban_china_province_path = '../external_datasets/urbanization_china_provinces.csv'
urban_china_province = pd.read_csv(urban_china_province_path)
urban_china_province.columns = ['Province_State', 'Urbanization']


# ========== URBANIZATION US STATES AND CHINA PROVINCES APPENDED ======== # for convenience
urban_all_provinces = urban_us_state.append(urban_china_province, ignore_index=True)


# ========== MEDIAN AGE BY COUNTRY========
# source: https://ourworldindata.org/grapher/median-age?year=2020&time=2020
median_age_per_countries_2020_path = '../external_datasets/median_age_per_countries_2020.csv'
median_age_per_countries_2020 = pd.read_csv(median_age_per_countries_2020_path)
median_age_per_countries_2020.columns = ['Country_Region', 'code', 'year', 'Median Age']
median_age_per_countries_2020 = median_age_per_countries_2020.drop(['code'], axis = 1)
median_age_per_countries_2020 = median_age_per_countries_2020[(median_age_per_countries_2020['year'] == 2020) & (median_age_per_countries_2020['Country_Region'] != 'United States')]
median_age_per_countries_2020 = median_age_per_countries_2020.drop(['year'], axis = 1)


# ========== US STATES MEDIAN AGE ========
# source: https://worldpopulationreview.com/states/median-age-by-state/
median_age_per_US_state_2020_path = '../external_datasets/median_age_per_US_state_2020.csv'
median_age_state_col_list = ['State', 'MedianAge']
median_age_per_US_state_2020 = pd.read_csv(median_age_per_US_state_2020_path, usecols = median_age_state_col_list)
median_age_per_US_state_2020.columns = ['Province_State', 'Median Age']


# ========== DEMOCRACY INDEX BY COUNTRY ========
# source: https://en.wikipedia.org/wiki/Democracy_Index#cite_note-index2019-7
democracy_index_by_country_path = '../external_datasets/democracy_index_by_country.csv'
democracy_col_list = ['Country', 'Score']
democracy_index_by_country = pd.read_csv(democracy_index_by_country_path, usecols = democracy_col_list)
democracy_index_by_country.columns = ['Country_Region', 'Democracy']


# ========== DAILY TESTS ADMINISTERED US STATES ========
d_tests_us_states_path = '../external_datasets/US_daily_tests_filtered.csv'
d_tests_us_states_df = pd.read_csv(d_tests_us_states_path)
target_cols = ['date', 'state', 'total']
d_tests_us_states_df = d_tests_us_states_df[target_cols]


# ========== POPULATION PER STATE ========
us_states_population_path = '../external_datasets/population_us_states.csv'
us_states_population = pd.read_csv(us_states_population_path)
us_states_population.columns = ['Province_State', 'State Pop']


# _Filling missing data_

Comments describe missing data and sources

In [217]:
# ========== GDP ========

# Virgin Islands source: https://www.macrotrends.net/countries/VIR/virgin-islands-us/gdp-per-capita
missing_GDP_per_US_territory_values = {
    'Province_State': ['American Samoa', 'Guam', 'Northern Mariana Islands', 'Puerto Rico', 'Virgin Islands'], 
    'GDP': [11466.69071, 35712.56214, 23258.67586, 31651.34815, 35938]
}

missing_GDP_per_US_territory_df = pd.DataFrame.from_dict(missing_GDP_per_US_territory_values)
gdp_all_provinces = gdp_all_provinces.append(missing_GDP_per_US_territory_df, ignore_index=True)


# ========== MEDIAN AGE ========

# Missing info (excluding Northern Mariana Islands) from median_age_per_countries_2020_path (were not listed as US states)
# Missing info for Northern Mariana Islands from https://en.wikipedia.org/wiki/List_of_countries_by_median_age
missing_median_age_per_US_territory_values = {
    'Province_State': ['American Samoa', 'Guam', 'Northern Mariana Islands', 'Puerto Rico', 'Virgin Islands'], 
    'Median Age': [22, 31.4, 33.6, 38.2, 42.2]
}

# Missing info from https://en.wikipedia.org/wiki/List_of_countries_by_median_age
# the 'US' entry is based on (37.0902, -95.7129) which is Kansas
# 'Holy See' - the Vatican. Was given the medain age in Italy
missing_median_age_per_country_values = {
    'Country_Region': ['East Timor', 'Congo (Kinshasa)', 'Congo (Brazzaville)', 'Dominica', 'Andorra', 'Kosovo', 'Liechtenstein', 
                       'San Marino', 'Monaco', 'Saint Kitts and Nevis', 'US', 'Holy See'], 
    'Median Age': [18.9, 18.6, 19.7, 33.5, 44.3, 29.1, 43.2, 44.4, 53.1, 35, 36.5, 47.9]
}

# Appending missing "Median Age" values 
missing_median_ages_per_US_state_df = pd.DataFrame.from_dict(missing_median_age_per_US_territory_values)
median_age_per_US_state_2020 = median_age_per_US_state_2020.append(missing_median_ages_per_US_state_df, ignore_index=True)

missing_median_ages_per_country_df = pd.DataFrame.from_dict(missing_median_age_per_country_values)
median_age_per_countries_2020 = median_age_per_countries_2020.append(missing_median_ages_per_country_df, ignore_index=True)



# ========== DEMOCRACY INDEX BY COUNTRY ========

### Missing info from https://en.wikipedia.org/wiki/List_of_freedom_indices & https://www.transparency.org/cpi2019
# Saint Vincent and the Grenadines (7.9, based on corruption between South Korea and Cabo Verde)
# Seychelles (7.9, based on corruption between Chile and Taiwan)
# Sao Tome and Principe (6.8, based on corruption between Croatia and Argentina)
# Maldives (4, based on corruption between Guinea and Mexico)
# Brunei (7.16, based on corruption of Malaysia)
# Somalia (1, based on corruption index last 3 years)
# Holy See (7.52, same as Italy)
# Grenada (7.3, based on corruption between Italy and Malaysia)
# Belize (7.02, based on list of freedoms indices)
# Dominica (7.78, ")
# Monaco (8.12, same as France)
# Barbados (6.2, based on CORRUPTION PERCEPTIONS INDEX 2019 https://www.transparency.org/cpi2019)
# Bahamas (6.4, ")
# Saint Lucia (5.5, ")
# Kosovo (3.6, ")
# Andorra (7.3, based on list of freedoms indices - like Grenada)
# Antigua and Barbuda (")
# Saint Kitts and Nevis (")
# Liechtenstein (")
# Western Sahara (3, based on list of freedoms indices)
# San Marino (7.5, considered as fairly democratic. Based on https://news.un.org/en/story/2013/04/435902-world-can-learn-san-marinos-democratic-system-says-un-chief)
missing_democracy_values = {
    'Country_Region': ['Saint Vincent and the Grenadines', 'Seychelles', 'Sao Tome and Principe', 'Maldives', 'Brunei', 
                       'Somalia', 'Holy See', 'Grenada', 'Belize', 'Dominica', 'Monaco', 'Barbados', 'Bahamas', 'Saint Lucia', 
                       'Kosovo', 'Andorra', 'Antigua and Barbuda', 'Saint Kitts and Nevis', 'Liechtenstein', 'Western Sahara', 'San Marino'],
    'Democracy': [7.9, 7.9, 6.8, 4, 7.16, 1, 7.52, 7.3, 7.02, 7.78, 8.12, 6.2, 6.4, 5.5, 3.6, 7.3, 7.3, 7.3, 7.3, 3, 7.5]
}

# Appending missing "Democracy" values 
missing_democracy_values_df = pd.DataFrame.from_dict(missing_democracy_values)
democracy_index_by_country = democracy_index_by_country.append(missing_democracy_values_df, ignore_index=True)

# _Translation Dictionaries_


Used to Standardize names in original df and external datasets

In [211]:
def standardize_us_states_names(df):
    df = df.replace({
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
})
    
    return df

In [212]:
def standardize_names(external_df):
    external_df = external_df.replace({
        'Bahamas, The': 'Bahamas',
        'Brunei Darussalam': 'Brunei',
        'Cape Verde': 'Cabo Verde',
        'Congo, Dem. Rep.': 'Congo (Kinshasa)',
        'Congo, Rep.': 'Congo (Brazzaville)',
        'Czech Republic':'Czechia',
        'Democratic Republic of the Congo': 'Congo (Kinshasa)',
        'Egypt, Arab Rep.': 'Egypt',
        'Eswatini': 'Swaziland',
        'Gambia, The': 'Gambia',
        'Iran, Islamic Rep.': 'Iran',
        'Ivory Coast': 'Cote d\'Ivoire',
        'Korea, Rep.': 'Korea, South',
        'Kyrgyz Republic': 'Kyrgyzstan',
        'Macedonia': 'North Macedonia',
        'Myanmar': 'Burma',
        'Palestine': 'West Bank and Gaza',
        'Republic of the Congo': 'Congo (Brazzaville)',
        'Russian Federation': 'Russia',
        'Slovak Republic': 'Slovakia',
        'South Korea': 'Korea, South',
        'South Korea[n 2]': 'Korea, South',        
        'St. Kitts and Nevis': 'Saint Kitts and Nevis',
        'St. Lucia': 'Saint Lucia',
        'St. Vincent and the Grenadines': 'Saint Vincent and the Grenadines',
        'Syrian Arab Republic': 'Syria',
        'Taiwan*': 'Taiwan',
        'Timor Leste': 'East Timor',
        'Timor-Leste': 'East Timor',
        'United States': 'US',
        'Venezuela, RB': 'Venezuela'
       })
    return external_df

In [213]:
# was used for standardizaing China province names
def china_check(urban_or_gdp):
    check = hopkins_conf[hopkins_conf['Country_Region']=='China']
    china_original_prov = set(check['Province_State'])
    china_gdp_prov = set(gdp_china_province.Province_State)

    orig_minus_gdp = china_original_prov.difference(china_gdp_prov)
    gdp_minus_orig = china_gdp_prov.difference(china_original_prov)

    china_original_prov = sorted(set(check['Province_State']))
    china_gdp_prov = sorted(set(gdp_china_province.Province_State))

    test_list = [china_gdp_prov, china_original_prov]
    for x, y in zip(*test_list): 
        print(x, y) # print(test_list)

    print('orig_minus_gdp: {}'.format(orig_minus_gdp))
    print('gdp_minus_orig: {}'.format(gdp_minus_orig))

## _Functions for adding US States daily tests data_

In [195]:
def add_index(df, row_name, gap=5):
    """
    Adds a row on level 1 of a df
    """
    idx = df.index
    previous_coor = (0,0)
    i = gap
    for coor, data in df.iterrows():
        coor = coor[0]
        if coor != previous_coor:
            idx = idx.insert(i, (coor, row_name))
            i += (gap + 1)
            previous_coor = coor
    return df.copy().reindex(idx)

def convert_dates(df_old):
    df_new = df_old.copy()
    df_new['date'] = df_new.apply(lambda row: datetime.datetime.strptime(str(row['date']), '%Y%m%d'), axis=1)
    return df_new


def add_daily_tests(df, tests_data):
    coordinates = set(df.index.get_level_values(level=0).tolist())
    common_dates = set(tests_data.index.get_level_values(level=0)).intersection(set(df.columns))
    counter = 0
    return_df = df.copy()
    for coor in coordinates:
        if counter % 50 == 0:
            print(counter)
        counter += 1
        if return_df.loc[coor].loc['data']['Country_Region']=='US':
            state = return_df.loc[coor].loc['data']['Province_State']
#             for (date,state) in tests_data.index:
            for date in common_dates:
                if (date,state) in tests_data.index:
#                 if date in common_dates:
#                     return_df.loc[coor,date]['d_tests'] = tests_data.loc[(date, state)]['d_tests']
                    return_df.loc[(coor, 'd_tests'),date] = tests_data.loc[(date, state)]['d_tests']
#                     print('State: {}, date: {}, d_tests: {}'.format(state, date,tests_data.loc[(date, state)]['d_tests']))
    return return_df 


In [219]:
# preprocessing for daily tests US states data

# convert dates formats
d_tests_us_states_df = convert_dates(d_tests_us_states_df)
# standardize names
d_tests_us_states_df = standardize_us_states_names(d_tests_us_states_df)
d_tests_us_states_df.sort_values(['state', 'date'], inplace=True)
# count daily from cumulated totals
d_tests_us_states_df['d_tests'] = d_tests_us_states_df.groupby(['state'])['total'].transform(lambda x: x.diff()) 
d_tests_us_states_df['date'] = d_tests_us_states_df.apply(lambda row: \
                            datetime.datetime.strftime(datetime.datetime.strptime(str(row['date']), '%Y-%m-%d %H:%M:%S'), '%-m/%-d/%Y'), axis=1)

us_states_population = standardize_us_states_names(us_states_population)
us_states_population = us_states_population.set_index('Province_State')


# add total tests divided by total population column
d_tests_us_states_df['total_tests_div_pop'] = d_tests_us_states_df.apply(lambda row: (row['total']/us_states_population.loc[row['state']]), axis=1)

# set multi index
d_tests_us_states_df = d_tests_us_states_df.set_index(['date','state'])

In [275]:
# STATE_POP_PKL_PATH = '../external_datasets/state_pop.pkl'
# with open(STATE_POP_PKL_PATH, 'wb') as file:
#     pickle.dump(new_hopkins_conf, file)
    
    
# TOTAL_TESTS_PKL_PATH = '../external_datasets/total_tests.pkl'
# with open(TOTAL_TESTS_PKL_PATH, 'wb') as file:
#     pickle.dump(new_hopkins_conf, file)

In [277]:
(d_tests_us_states_df.xs('Alabama', level='state').loc[max_date])['total_tests_div_pop']
# df.xs('Ai', level='name', drop_level=False)
# d_tests_us_states_df

0.003042104264880889

In [143]:
d_tests_us_states_df.iloc[d_tests_us_states_df.index.get_level_values('state') == 'Missouri']



,,total,d_tests
date,state,,
3/7/2020,Missouri,0.0,NaN
3/8/2020,Missouri,1.0,1.0
3/9/2020,Missouri,1.0,0.0
3/10/2020,Missouri,1.0,0.0
3/11/2020,Missouri,1.0,0.0
3/12/2020,Missouri,65.0,64.0
3/13/2020,Missouri,73.0,8.0
3/14/2020,Missouri,94.0,21.0
3/15/2020,Missouri,127.0,33.0


## _this is where the magic happens_

In [281]:
PROVINCE_STATE = 'Province_State'
COUNTRY_REGION = 'Country_Region'
DAILY_TESTS = 'd_tests'
MAX_DATE = '4/7/2020'


def add_gdp_urban(original_df, args_datasets):
    
    # import argument dataframes from input dict
    gdp_country = args_datasets['gdp_country']
    gdp_all_provinces = args_datasets['gdp_all_provinces'] 
    urban_country = args_datasets['urban_country']
    urban_all_provinces = args_datasets['urban_all_provinces'] 
    median_age_per_countries_2020 = args_datasets['median_age_per_countries_2020']
    median_age_per_US_state_2020 = args_datasets['median_age_per_US_state_2020']
    democracy_index_by_country = args_datasets['democracy_index_by_country']
    d_tests_us_states_df = args_datasets['d_tests_us_states_df']
    us_states_population = args_datasets['us_states_population']
    

    
    # standardizing region names
    result = standardize_names(original_df)
    gdp_country = standardize_names(gdp_country)
    urban_country = standardize_names(urban_country)
    median_age_per_countries_2020 = standardize_names(median_age_per_countries_2020)
    democracy_index_by_country = standardize_names(democracy_index_by_country)

    
    # setting indices for more convenient access in lambda funtion below
    gdp_country = gdp_country.set_index(COUNTRY_REGION)
    gdp_all_provinces = gdp_all_provinces.set_index(PROVINCE_STATE)
    urban_country = urban_country.set_index(COUNTRY_REGION)
    urban_all_provinces = urban_all_provinces.set_index(PROVINCE_STATE)
    median_age_per_US_state_2020 = median_age_per_US_state_2020.set_index(PROVINCE_STATE)
    median_age_per_countries_2020 = median_age_per_countries_2020.set_index(COUNTRY_REGION)
    democracy_index_by_country = democracy_index_by_country.set_index(COUNTRY_REGION)


#     if there is no state data, take country data, and if there is also no country data, put NaN
    result['GDP'] = result.apply(lambda row: gdp_all_provinces.loc[row[PROVINCE_STATE],'GDP'] \
                                     if row[PROVINCE_STATE] in list(gdp_all_provinces.index) \
                                     else (gdp_country.loc[row[COUNTRY_REGION],'GDP'] \
                                           if row[COUNTRY_REGION] in list(gdp_country.index) \
                                           else np.NaN), axis=1) 
    
    result['Urbanization'] = result.apply(lambda row: urban_all_provinces.loc[row[PROVINCE_STATE],'Urbanization'] \
                                              if row[PROVINCE_STATE] in list(urban_all_provinces.index) \
                                              else (urban_country.loc[row[COUNTRY_REGION],'Urbanization'] \
                                                    if row[COUNTRY_REGION] in list(urban_country.index) \
                                                    else np.NaN), axis=1)
    
    result['Median Age'] = result.apply(lambda row: median_age_per_US_state_2020.loc[row[PROVINCE_STATE],'Median Age'] \
                                            if row[PROVINCE_STATE] in list(median_age_per_US_state_2020.index) \
                                            else (median_age_per_countries_2020.loc[row[COUNTRY_REGION], 'Median Age'] \
                                                  if row[COUNTRY_REGION] in list(median_age_per_countries_2020.index) \
                                                  else np.NaN), axis=1)
    
    result['Democracy'] = result.apply(lambda row: democracy_index_by_country.loc[row[COUNTRY_REGION], 'Democracy'] \
                                           if row[COUNTRY_REGION] in list(democracy_index_by_country.index) \
                                           else np.NaN, axis=1)
    
    
    result['State Population'] = result.apply(lambda row: us_states_population.loc[row[PROVINCE_STATE]]['State Pop'] \
                                              if row[PROVINCE_STATE] in list(us_states_population.index) \
                                              else np.NaN, axis = 1)
    
    result['Total Tests'] = result.apply(lambda row: (d_tests_us_states_df.xs(row[PROVINCE_STATE], level='state').loc[max_date])['total']
                                              if row[PROVINCE_STATE] in list(us_states_population.index) \
                                              else np.NaN, axis = 1)
    
    
    result['Tests \ Pop'] = result.apply(lambda row: (d_tests_us_states_df.xs(row[PROVINCE_STATE], level='state').loc[max_date])['total_tests_div_pop']
                                              if row[PROVINCE_STATE] in list(us_states_population.index) \
                                              else np.NaN, axis = 1)
    
    
    
    # adding daily_tests administered field to index level 1
    result = add_index(result,DAILY_TESTS)

    # adding tests administered per data per US State
    result = add_daily_tests(result, d_tests_us_states_df)
    
    # reordering columns
    new_columns = list(result.columns)

    prev_GDP = new_columns.index('GDP')
    prev_urban = new_columns.index('Urbanization')
    prev_median_age = new_columns.index('Median Age')
    prev_democracy = new_columns.index('Democracy')
    prev_state_pop = new_columns.index('State Population')
    prev_total_tests = new_columns.index('Total Tests')
    prev_tests_div_pop = new_columns.index('Tests \ Pop')

    new_GDP = new_columns.index(COUNTRY_REGION) + 1
    new_urban = new_GDP + 1
    new_median_age = new_urban + 1
    new_democracy = new_median_age + 1
    new_state_pop = new_democracy + 1
    new_total_tests = new_state_pop + 1
    new_tests_div_pop = new_total_tests +1

    new_columns.insert(new_GDP, new_columns.pop(prev_GDP))
    new_columns.insert(new_urban, new_columns.pop(prev_urban))
    new_columns.insert(new_median_age, new_columns.pop(prev_median_age))
    new_columns.insert(new_democracy, new_columns.pop(prev_democracy))
    new_columns.insert(new_state_pop, new_columns.pop(prev_state_pop))
    new_columns.insert(new_total_tests, new_columns.pop(prev_total_tests))
    new_columns.insert(new_tests_div_pop, new_columns.pop(prev_tests_div_pop))

    return result[new_columns]


In [282]:
args_datasets = {
    'gdp_country': gdp_country, 
    'gdp_all_provinces': gdp_all_provinces, 
    'urban_country': urban_country, 
    'urban_all_provinces': urban_all_provinces, 
    'median_age_per_countries_2020': median_age_per_countries_2020, 
    'median_age_per_US_state_2020': median_age_per_US_state_2020, 
    'democracy_index_by_country': democracy_index_by_country,
    'd_tests_us_states_df': d_tests_us_states_df,
    'us_states_population': us_states_population
}
new_hopkins_conf = add_gdp_urban(hopkins_conf, args_datasets)

new_hopkins_conf

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050


Province_State Country_Region          GDP  \
coordinate           information                                               
(-41.4545, 145.9707) data               Tasmania      Australia  57373.68668   
                     avg_d_RH                NaN            NaN          NaN   
                     avg_d_precip            NaN            NaN          NaN   
                     avg_d_tmp               NaN            NaN          NaN   
                     avg_d_wind              NaN            NaN          NaN   
...                                          ...            ...          ...   
(64.9631, -19.0208)  avg_d_RH                NaN            NaN          NaN   
                     avg_d_precip            NaN            NaN          NaN   
                     avg_d_tmp               NaN            NaN          NaN   
                     avg_d_wind              NaN            NaN          NaN   
                     d_tests                 NaN            NaN          NaN   

                                   Urbanization  Median Age Democracy  \
coordinate           information                                        
(-41.4545, 145.9707) data                86.012   37.900002      9.09   
                     avg_d_RH               NaN         NaN       NaN   
                     avg_d_precip           NaN         NaN       NaN   
                     avg_d_tmp              NaN         NaN       NaN   
                     avg_d_wind             NaN         NaN       NaN   
...                                         ...         ...       ...   
(64.9631, -19.0208)  avg_d_RH               NaN         NaN       NaN   
                     avg_d_precip           NaN         NaN       NaN   
                     avg_d_tmp              NaN         NaN       NaN   
                     avg_d_wind             NaN         NaN       NaN   
                     d_tests                NaN         NaN       NaN   

                                   State Population  Total Tests  Tests \ Pop  \
coordinate           information                                                
(-41.4545, 145.9707) data                       NaN          NaN          NaN   
                     avg_d_RH                   NaN          NaN          NaN   
                     avg_d_precip               NaN          NaN          NaN   
                     avg_d_tmp                  NaN          NaN          NaN   
                     avg_d_wind                 NaN          NaN          NaN   
...                                             ...          ...          ...   
(64.9631, -19.0208)  avg_d_RH                   NaN          NaN          NaN   
                     avg_d_precip               NaN          NaN          NaN   
                     avg_d_tmp                  NaN          NaN          NaN   
                     avg_d_wind                 NaN          NaN          NaN   
                     d_tests                    NaN          NaN          NaN   

                                   avg_m_tmp  ...  3/29/2020  3/30/2020  \
coordinate           information              ...                         
(-41.4545, 145.9707) data          15.105195  ...       66.0       66.0   
                     avg_d_RH            NaN  ...       85.0       80.0   
                     avg_d_precip        NaN  ...        0.0        0.0   
                     avg_d_tmp           NaN  ...       15.6       14.8   
                     avg_d_wind          NaN  ...       14.5       24.9   
...                                      ...  ...        ...        ...   
(64.9631, -19.0208)  avg_d_RH            NaN  ...       61.0       63.0   
                     avg_d_precip        NaN  ...        0.0        0.0   
                     avg_d_tmp           NaN  ...        6.5        6.2   
                     avg_d_wind          NaN  ...       31.3       31.4   
                     d_tests             NaN  ...        NaN        NaN   

             

## _Some tests_

In [283]:
age_test = pd.DataFrame(new_hopkins_conf.xs('data', level='information', axis=0))
age_test = age_test[age_test['Median Age'].isna()]
age_test

,Province_State,Country_Region,GDP,Urbanization,Median Age,Democracy,State Population,Total Tests,Tests \ Pop,avg_m_tmp,...,3/29/2020,3/30/2020,3/31/2020,4/1/2020,4/2/2020,4/3/2020,4/4/2020,4/5/2020,4/6/2020,4/7/2020
coordinate,,,,,,,,,,,,,,,,,,,,,


In [284]:
dem_test = pd.DataFrame(new_hopkins_conf.xs('data', level='information', axis=0))
dem_test = dem_test[dem_test['Democracy'].isna()]
dem_test

,Province_State,Country_Region,GDP,Urbanization,Median Age,Democracy,State Population,Total Tests,Tests \ Pop,avg_m_tmp,...,3/29/2020,3/30/2020,3/31/2020,4/1/2020,4/2/2020,4/3/2020,4/4/2020,4/5/2020,4/6/2020,4/7/2020
coordinate,,,,,,,,,,,,,,,,,,,,,


In [285]:
check = new_hopkins_conf[new_hopkins_conf['Urbanization'].isna()]
check = check[check['Country_Region'].notna()]
check

,,Province_State,Country_Region,GDP,Urbanization,Median Age,Democracy,State Population,Total Tests,Tests \ Pop,avg_m_tmp,...,3/29/2020,3/30/2020,3/31/2020,4/1/2020,4/2/2020,4/3/2020,4/4/2020,4/5/2020,4/6/2020,4/7/2020
coordinate,information,,,,,,,,,,,,,,,,,,,,,


In [286]:
check = new_hopkins_conf[new_hopkins_conf['GDP'].isna()]
check = check[check['Country_Region'].notna()]
check

,,Province_State,Country_Region,GDP,Urbanization,Median Age,Democracy,State Population,Total Tests,Tests \ Pop,avg_m_tmp,...,3/29/2020,3/30/2020,3/31/2020,4/1/2020,4/2/2020,4/3/2020,4/4/2020,4/5/2020,4/6/2020,4/7/2020
coordinate,information,,,,,,,,,,,,,,,,,,,,,


In [287]:
gdp = len(new_hopkins_conf[new_hopkins_conf[COUNTRY_REGION]=='US']['GDP'].unique())
print('Unique GDP values for US states: {}'.format(gdp))

urb = len(new_hopkins_conf[new_hopkins_conf[COUNTRY_REGION]=='US']['Urbanization'].unique())
print('Unique Urbanization values for US states: {}'.format(urb))

Unique GDP values for US states: 55
Unique Urbanization values for US states: 55


In [288]:
# Checking that our starting data has no nan country_region entries in 'data' rows
check_old = pd.DataFrame(hopkins_conf.xs('data', level='information', axis=0)).shape
print('Original df data rows dimensions: {}'.format(check_old))

check_new = pd.DataFrame(new_hopkins_conf.xs('data', level='information', axis=0)).shape
print('New df data rows dimensions: {}'.format(check_new))


Original df data rows dimensions: (1070, 95)
New df data rows dimensions: (1070, 100)


In [289]:
def test_d_tests(df, tests_data):
    tests_data_dates = tests_data.index.get_level_values(level=0)
    common_dates = set(tests_data_dates).intersection(set(df.columns))
    printed = []
    dict = {}
    coords = set(df.index.get_level_values(level=0).tolist())
    for coord in coords:
        if df.loc[coord].loc['data']['Country_Region']=='US':
            state = df.loc[coord].loc['data']['Province_State']
            if not (state in printed):
                printed.append(state)
#                 print(state)
#                 print(coord)
#                 print(df.loc[coord].loc['d_tests'][common_dates].tolist())
                dict[state] = pd.DataFrame(df.loc[coord][common_dates])
    return dict

test_dict = test_d_tests(new_hopkins_conf, d_tests_us_states_df)


In [290]:
test_state = 'California'

test_dict[test_state]

,3/8/2020,3/19/2020,3/9/2020,4/5/2020,3/29/2020,3/23/2020,3/27/2020,3/16/2020,3/12/2020,3/11/2020,...,3/20/2020,3/6/2020,3/21/2020,3/17/2020,3/7/2020,3/14/2020,3/28/2020,4/4/2020,3/24/2020,4/1/2020
information,,,,,,,,,,,,,,,,,,,,,
data,5.0,9.0,7.00,103.00,53.00,12.00,30.0,5.0,7.0,7.0,...,9.0,5.0,9.0,5.0,5.00,7.00,41.00,97.00,20.00,78.0
avg_d_RH,94.0,77.0,86.00,93.00,91.00,88.00,62.0,97.0,39.0,62.0,...,76.0,66.0,61.0,93.0,95.00,97.00,83.00,85.00,94.00,74.0
avg_d_precip,3.3,0.0,0.76,84.07,3.81,1.02,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,23.88,22.35,6.35,26.92,18.54,0.0
avg_d_tmp,0.3,4.1,1.70,-1.20,0.80,2.10,0.3,0.8,16.1,11.6,...,5.9,7.9,8.4,1.4,0.30,2.40,0.10,0.50,-1.20,3.2
avg_d_wind,7.6,3.2,3.50,9.30,6.00,15.60,8.2,0.4,7.6,5.3,...,3.4,18.2,4.0,2.4,15.20,8.80,10.40,17.20,15.90,12.9
d_tests,19.0,1119.0,254.00,4833.00,1065.00,13560.00,873.0,7107.0,45.0,250.0,...,1776.0,7.0,1041.0,148.0,9.00,50.00,10933.00,31900.00,1254.00,673.0


In [291]:
d_tests_us_states_df.iloc[d_tests_us_states_df.index.get_level_values('state') == test_state].T


date,3/4/2020,3/5/2020,3/6/2020,3/7/2020,3/8/2020,3/9/2020,3/10/2020,3/11/2020,3/12/2020,3/13/2020,...,4/5/2020,4/6/2020,4/7/2020,4/8/2020,4/9/2020,4/10/2020,4/11/2020,4/12/2020,4/13/2020,4/14/2020
state,California,California,California,California,California,California,California,California,California,California,...,California,California,California,California,California,California,California,California,California,California
total,515.000000,515.000000,522.000000,531.000000,550.000000,804.00000,823.000000,1073.000000,1118.000000,1118.000000,...,131533.000000,132431.000000,145329.000000,158864.000000,177600.000000,178763.000000,185276.000000,203528.000000,204082.000000,215408.000000
d_tests,NaN,0.000000,7.000000,9.000000,19.000000,254.00000,19.000000,250.000000,45.000000,0.000000,...,4833.000000,898.000000,12898.000000,13535.000000,18736.000000,1163.000000,6513.000000,18252.000000,554.000000,11326.000000
total_tests_div_pop,0.000013,0.000013,0.000013,0.000013,0.000014,0.00002,0.000021,0.000027,0.000028,0.000028,...,0.003329,0.003352,0.003678,0.004021,0.004495,0.004524,0.004689,0.005151,0.005165,0.005452


In [292]:
# how many unique values does a given date have? 
# we expect the number of states or less because of repetitions
test_date = '4/5/2020'
idx = pd.IndexSlice
len(new_hopkins_conf.loc[idx[:, 'd_tests'], :][test_date].unique())


55

In [293]:
new_hopkins_conf[new_hopkins_conf['Country_Region']=='US']

,,Province_State,Country_Region,GDP,Urbanization,Median Age,Democracy,State Population,Total Tests,Tests \ Pop,avg_m_tmp,...,3/29/2020,3/30/2020,3/31/2020,4/1/2020,4/2/2020,4/3/2020,4/4/2020,4/5/2020,4/6/2020,4/7/2020
coordinate,information,,,,,,,,,,,,,,,,,,,,,
"(13.4443, 144.7937)",data,Guam,US,35712.56214,94.780,31.4,7.96,165718.0,650.0,0.003922,26.877922,...,56.0,58.0,69.0,77.0,82.0,84.0,93.0,112.0,113.0,121.0
"(18.2208, -66.5901)",data,Puerto Rico,US,31651.34815,93.578,38.2,7.96,3193694.0,5507.0,0.001724,25.748052,...,127.0,174.0,239.0,286.0,316.0,316.0,452.0,475.0,513.0,573.0
"(18.3358, -64.8963)",data,Virgin Islands,US,35938.00000,95.721,42.2,7.96,104914.0,285.0,0.002717,25.875000,...,23.0,30.0,30.0,30.0,30.0,37.0,40.0,42.0,43.0,43.0
"(19.60121157, -155.5210167)",data,Hawaii,US,64096.00000,91.900,38.9,7.96,1415872.0,13542.0,0.009564,14.373611,...,10.0,12.0,15.0,15.0,18.0,20.0,22.0,22.0,23.0,23.0
"(20.86399628, -156.56890969999995)",data,Hawaii,US,64096.00000,91.900,38.9,7.96,1415872.0,13542.0,0.009564,22.872727,...,16.0,20.0,25.0,25.0,26.0,36.0,38.0,43.0,44.0,48.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(48.29575866, -114.0520569)",data,Montana,US,46609.00000,55.900,39.8,7.96,1068778.0,6985.0,0.006536,0.163636,...,7.0,9.0,11.0,14.0,18.0,18.0,20.0,24.0,25.0,31.0
"(48.48171488, -121.766131)",data,Washington,US,74182.00000,84.100,37.6,7.96,7614893.0,92434.0,0.012139,5.171429,...,99.0,101.0,121.0,128.0,143.0,153.0,161.0,161.0,161.0,166.0
"(48.82227976, -121.7490018)",data,Washington,US,74182.00000,84.100,37.6,7.96,7614893.0,92434.0,0.012139,5.274026,...,116.0,116.0,139.0,139.0,175.0,182.0,199.0,224.0,228.0,238.0


## _Writing results to pickle file_

In [294]:
with open(RESULT_PATH, 'wb') as file:
    pickle.dump(new_hopkins_conf, file)

## _Checking that pickle was written correctly and loads correctly_

In [295]:
test = pd.DataFrame()
with open(RESULT_PATH, 'rb') as file:
    test = pd.read_pickle(file)    
test.head

<bound method NDFrame.head of                                   Province_State Country_Region          GDP  \
coordinate           information                                               
(-41.4545, 145.9707) data               Tasmania      Australia  57373.68668   
                     avg_d_RH                NaN            NaN          NaN   
                     avg_d_precip            NaN            NaN          NaN   
                     avg_d_tmp               NaN            NaN          NaN   
                     avg_d_wind              NaN            NaN          NaN   
...                                          ...            ...          ...   
(64.9631, -19.0208)  avg_d_RH                NaN            NaN          NaN   
                     avg_d_precip            NaN            NaN          NaN   
                     avg_d_tmp               NaN            NaN          NaN   
                     avg_d_wind              NaN            NaN          NaN   
          

In [296]:
len(test.loc[idx[:, 'd_tests'], :][test_date].unique())

55